# Cross-correlation loss

In [1]:
import torch

from lossfunctions.cross_correlation import CrossCorrelationLoss

In [2]:
loss = CrossCorrelationLoss(8)

coeffs = torch.randn(45).to('mps').unsqueeze(0)
coeffs2 = torch.randn(45).to('mps').unsqueeze(0)

loss(coeffs,coeffs)/torch.sqrt(torch.sum(torch.square(coeffs)) * torch.sum(torch.square(coeffs)))

tensor(1., device='mps:0')

In [4]:
torch.sum(torch.square(coeffs)) + torch.sum(torch.square(coeffs)) - 2*loss(coeffs,coeffs)

tensor(0., device='mps:0')

# Data loader

In [1]:
from dataloader.load_dt_simulated import load_dt_simulated_dataset

In [2]:
data = load_dt_simulated_dataset(dataset_size=100, number_of_fibre_populations=2, max_degree=8,
                          fibre_population_eigenvalues=None, number_of_data_points=90, b_value=1000,
                          b_0_signal=3000, include_b_0=False,
                          signal_to_noise_ratio=30, noise_type='rician',
                          noise_generator_seed=1,
                          gradient_generator_seed=1,
                          fibre_orientation_generator_seed=1, planar=False)

Generating fibre orientations and volume fractions...
[array([[ 0.27298068, -0.75481445, -0.59643666],
       [-0.75313675,  0.5551685 ,  0.35294896]]), array([[ 0.24279884, -0.47817368,  0.84403712],
       [-0.34045643, -0.36048254, -0.86841336]]), array([[ 0.66024121, -0.73518671,  0.15356447],
       [-0.71911451,  0.67816572,  0.15154398]]), array([[-0.417361  ,  0.32339033,  0.84925172],
       [-0.58146605, -0.61374998, -0.53404887]]), array([[ 0.8200692 ,  0.54940183,  0.16013787],
       [ 0.30562602, -0.18266288,  0.93446616]]), array([[ 0.12319798,  0.91352787, -0.38767136],
       [-0.69348122, -0.36200853,  0.62292345]]), array([[-0.77413749,  0.46655183,  0.42783238],
       [-0.75676794,  0.08192617,  0.6485294 ]]), array([[ 0.98665063,  0.1138056 , -0.11648529],
       [-0.75132162,  0.6314812 , -0.19169592]]), array([[-0.08399333,  0.51857147, -0.85089879],
       [-0.49118615, -0.78142682,  0.38484841]]), array([[ 0.61898402, -0.77509971,  0.1268039 ],
       [-0.7495

In [9]:
data[1][0]

(array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000]),
 array([[ 0.36353657, -0.79057113, -0.61636162,  0.47329009, -0.82269029,
          0.89697745, -0.94988454, -0.16577181, -0.81933594, -0.8108305 ,
          0.87454161,  0.91075499, -0.29634718,  0.07737383, -0.0756303 ,
          0.18488546,  0.53113688, -0.67804166,  0.52939376, -0.0155802 ,
          0.0529998 , -0.9090384 ,  0.12865447,  0.19786134, -0.71456792,
          0.5073

# Multiprocessing

In [1]:
import multiprocessing

In [2]:
print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  12


In [2]:
from utils.test_worker import main, hash_word

main()

<class 'numpy.ndarray'>
<class 'list'>
[(0.867965290007741, 1.867965290007741), (0.9612228278389066, 1.9612228278389066), (0.4315516021038266, 1.4315516021038266), (0.6099616459337621, 1.6099616459337622), (0.5633595493067229, 1.563359549306723), (0.8213059279304693, 1.8213059279304693), (0.5137782895464704, 1.5137782895464704), (0.3350059090860372, 1.3350059090860373), (0.6381156476287936, 1.6381156476287937), (0.004493453783756318, 1.0044934537837564), (0.8132633836909182, 1.8132633836909182), (0.7571507751762853, 1.7571507751762852), (0.32768808611037814, 1.3276880861103781), (0.5260313462572175, 1.5260313462572175), (0.10993230210640059, 1.1099323021064005), (0.5311337427672019, 1.531133742767202), (0.6955622680407046, 1.6955622680407045), (0.3402876229821604, 1.3402876229821605), (0.8170573296848797, 1.8170573296848795), (0.08981667158642426, 1.0898166715864241), (0.7375437502236406, 1.7375437502236406), (0.5998663444396414, 1.5998663444396413), (0.8888779825224418, 1.888877982522

In [3]:
from functools import partial

In [5]:
def xfunc(a,b):
    print('a ',a)
    print('b ',b)

In [6]:

partfunc = partial(xfunc, a=5)

In [7]:
partfunc(6)

TypeError: xfunc() got multiple values for argument 'a'